In [3]:
# import packages
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import chromedriver_binary
import random
from datetime import timedelta, date
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException

In [4]:
#parametros e site
params = {'login':'04222526380',
          'password':'339715'}

driver = webdriver.Edge() # escolhendo o driver
driver.get("https://radar.serpro.gov.br/")# select site


In [5]:
def login():
  currentUrl = driver.current_url;
  if currentUrl == "https://radar.serpro.gov.br/main.html#/login":
    try:
      element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'usuario')))       
      element.send_keys(params['login'])
      element = driver.find_element(By.ID, "password")
      element.send_keys(params['password'])
      element = driver.find_element_by_id('Entrar')
      element.click()
      print("login feito")
      #acess the search of the taxs
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
      'body > div.bg > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, 
      'body > div > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown.open > ul > div > li:nth-child(1) > ul > li:nth-child(1) > a')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID,'abaBuscaAvancada')))
      element.click()
      print("Acesso Menu realizado")
    except Exception as e:
      print("Não foi possível fazer login")
      print(e)
  print("Operação completa")
  
login() 

C:\Users\Filipe\AppData\Local\Temp\ipykernel_8816\3153827461.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id('Entrar')


login feito
Acesso Menu realizado
Operação completa


In [31]:
df_abril = pd.read_excel("ResultadoConsultaInfracoes_abril.xlsx", skiprows=5)
df_abril.columns = df_abril.iloc[0]
df_abril = df_abril[1:]
df_teste = df_abril['N. Auto'].head(10)
for i in df_teste:
    print(str(i))    

c:\Users\Filipe\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ER00010639
ER00010638
ER00010637
ER00010636
ER00010635
ER00010634
ER00010633
ER00010154
ER00010632
ER00010631


In [38]:
sne_list = []
local_list = []
data_hora_list = []
codigo_list = []
agente_list = []



def souce_finn(auto):
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'codigo')))
        element.clear()
        element.send_keys(str(auto))
        element = driver.find_element_by_id('btnBuscaAvancada')
        element.click()
        print("Pesquisa Realizada")
        time.sleep(2)
        table_id = driver.find_element_by_css_selector('#vm\.id > tbody')
        rows = table_id.find_elements(By.TAG_NAME, "tr") 
        for row in rows:
            col = row.find_elements(By.TAG_NAME, "td")[11] 
            sne_list.append(col.text)
        element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#vm\.id > tbody > tr > td:nth-child(2) > a')))
        element.click()
        # time.sleep(2)
        WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body > div.modal.ng-isolate-scope.in > div > div > div.modal-header.ng-scope > h4')))
        local = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[1]')[0].text
        data_hora = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[5]')[0].text
        codigo = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[1]')[0].text
        agente = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[7]')[0].text
        print(local)
        local_list.append(local)
        print(data_hora)
        data_hora_list.append(data_hora)
        print(codigo)
        codigo_list.append(codigo)
        print(agente)
        agente_list.append(agente)
        driver.find_element_by_css_selector("#btnFecharDetalhe").click()
    except Exception as e:
      print("Acesso Pesquisa")
      print(e)
      print("Operação completa")

# souce_fine()


In [ ]:
for i in df_teste:
    print(i)
    souce_finn(i)

In [57]:
matricula = [matricula.split(': ', 1)[1] for matricula in agente_list]
matricula

['10750',
 '10750',
 '10750',
 '10750',
 '10750',
 '10750',
 '10750',
 '43484',
 '10750',
 '10750']

In [58]:
# separando as strings de local
local = [local.split(': ', 1)[1] for local in local_list]
local

['Av. Maranhao, prox ao n. 2141',
 'AVENIDA JOSE FRANCISCO DE ALMEIDA NETO, N. 2767',
 'Av. Maranhao, prox ao n. 2141',
 'AVENIDA JOSE FRANCISCO DE ALMEIDA NETO, N. 2767',
 'AV. PRESIDENTE KENNEDY, PROX AO COND TERRA DOS PASSAROS',
 'AV. PRESIDENTE KENNEDY, PROX AO COND TERRA DOS PASSAROS',
 'AVENIDA JOSE FRANCISCO DE ALMEIDA NETO N. 2767',
 'Av. Pres. Kennedy com a Av. Dom Severino',
 'AV. PRESIDENTE KENNEDY, PROX AO COND TERRA DOS PASSAROS',
 'Av. Miguel Rosa, prox. 25 BC']

In [60]:
codigos = [codigo.replace(' - ', '') for codigo in codigo_list]
codigo = [codigo.split(': ', 1)[1] for codigo in codigos]
codigo

['74630',
 '74550',
 '74550',
 '74630',
 '74550',
 '74550',
 '74630',
 '75870',
 '74550',
 '74630']

In [68]:
hora_data = [hora_data.split(': ', 1)[1] for hora_data in data_hora_list]
data = [data.split(' ', 1)[0] for data in hora_data]
data

['30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022',
 '30/04/2022']

In [71]:
hora = [hora.split(' ', 1)[1] for hora in hora_data]
hora

['23:58:47',
 '23:50:05',
 '23:36:54',
 '23:18:13',
 '23:11:59',
 '23:10:34',
 '23:08:55',
 '23:05:40',
 '22:55:33',
 '22:37:30']